# Auto-Fix with GeoFix

Automatically repair invalid geometries and save corrected datasets.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmmarYasser455/GeoFix/blob/main/examples/02_auto_fix.ipynb)

In [ ]:
# !pip install geofix -q

In [ ]:
import geopandas as gpd
import geofix
from shapely.geometry import Polygon

## 1. Create Dataset with Invalid Geometries

In [ ]:
# Self-intersecting polygons
features = [
    Polygon([(0, 0), (1, 0), (1, 1), (0, 1)]),       # Valid square
    Polygon([(2, 0), (4, 2), (4, 0), (2, 2)]),       # Invalid bowtie
    Polygon([(5, 0), (7, 2), (7, 0), (5, 2)]),       # Invalid bowtie
    Polygon([(8, 8), (9, 8), (9, 9), (8, 9)]),       # Valid square
]

gdf = gpd.GeoDataFrame(
    {"id": [1, 2, 3, 4], "status": ["ok", "broken", "broken", "ok"]},
    geometry=features,
    crs="EPSG:4326",
)

gdf.to_file("broken_data.shp")
print(f"Valid before fix: {gdf.geometry.is_valid.sum()}/{len(gdf)}")

## 2. Analyse Before Fixing

In [ ]:
before = geofix.validate("broken_data.shp")
print("BEFORE:")
print(before.summary())

## 3. Auto-Fix

GeoFix uses `buffer(0)` to repair self-intersections — a standard GIS technique.

In [ ]:
after = geofix.fix("broken_data.shp", "repaired_data.gpkg")
print("AFTER:")
print(after.summary())

## 4. Verify the Fix

In [ ]:
fixed_gdf = gpd.read_file("repaired_data.gpkg")
print(f"Valid after fix: {fixed_gdf.geometry.is_valid.sum()}/{len(fixed_gdf)}")
print(f"\nQuality score: {before['quality_score']} → {after['quality_score']}")

## How Auto-Fix Works

| Fix Type | Technique | When Used |
|----------|-----------|----------|
| `buffer(0)` | Repairs self-intersections | Invalid polygons |
| `make_valid()` | Shapely's built-in fixer | Complex topology |

For overlaps, boundary violations, and road conflicts, use the full GeoFix pipeline:

```bash
geofix chat
```